In [8]:
import numpy as np
from scipy.linalg import expm,logm
from scipy.integrate import simps
from tqdm import tqdm

Fonctions intermédiaires

In [9]:
#pour virer les termes égaux à 0
def apply_threshold(matrix, threshold):
    result = matrix.copy()
    
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if abs(result[i, j]) < threshold:
                result[i, j] = 0.0
    return result

#pour print une matrice de manière sympa
def printArray(arr):
    for row in arr:
        for item in row:
            print("{:8.3f}".format(item), end = " ")
        print("")

## Fonctions utiles pour les calculs

In [10]:
#calcul de Tr(B^T A)
def frobenius_inner_product(A, B):
    return np.sum(A * B)

#matrice élémentaire
def elementary_matrix(k, l, n):
    E = np.zeros((n,n))
    E[k, l] = 1
    return E

def energie(A,x,y):
  N = len(x)  # Nombre de vecteurs x_i et y_i
  diff = sum([np.linalg.norm(A@x_i - y_i) for (x_i,y_i) in zip(x,y)])
  return diff/N

#intégrande pour le calcul de la dérivée de Fréchet
def integrand(A,k,l,E_kl,t):
  return np.array([(expm((1-tau)*A))@E_kl@(expm(tau*A)) for tau in t])

#dérivée sens fréchet
def frechet(A,k,l):
  n=A.shape[0]
  E_kl=elementary_matrix(k,l,n)
  t = np.linspace(0, 1, 101)
  integ = integrand(A,k,l,E_kl,t)
  A_reshaped = integ.reshape((101,n*n))
  return simps(A_reshaped.T,t).reshape((n,n))

#somme des x_i y_i^T pour calculer X et S
def calcul_vecteurs(x_vectors,y_vectors):
    n = len(x_vectors)
    result = np.zeros_like(x_vectors[0].reshape(-1, 1) @ y_vectors[0].reshape(1, -1))

    for i in range(n):
        xi = x_vectors[i]
        yi = y_vectors[i]
        result += xi.reshape(-1, 1) @ yi.reshape(1, -1)
    return result

def calcul_X(x):
  return calcul_vecteurs(x,x)

def calcul_Y(x,y):
  return calcul_vecteurs(y,x)

#terme correspondant à la dérivée de la régression
def regression_term(A,X,S):
  return (expm(A)@X-S)

def grad_kl(A,k,l,reg_term):
  return frobenius_inner_product(frechet(A,k,l),reg_term)

#matrice qui recense l'intégralité de l'évolution des coeffs de A
def grad_matrix(A,X,S):
  reg_term=regression_term(A,X,S)
  n=A.shape[0]
  grad_matrix=np.zeros((n,n))
  for k in range(n):
    for l in range(n):
      grad_matrix[k,l]=grad_kl(A,k,l,reg_term)
  return grad_matrix

def matrix_log(A, num_terms):
    n = A.shape[0]
    identity = np.eye(n)
    term = A - identity
    result = term
    for i in range(2, num_terms + 1):
        term = np.matmul(term, (A - identity) / i)
        result += term
    return result

## Calcul de exp(A(s))

In [31]:
#calcule A(T) avec un intervalle de temps dt
def compute_A(A0, X, S,x,y, dt, T, B):
    A = A0.copy()
    t = 0    
    iteration = 0
    it_total=int(T/dt)
    
    while t < T:
        dA_dt = -grad_matrix(A, X, S)
        A += dA_dt * dt
        t += dt
        iteration += 1
        
        if iteration % 50 == 0:
            print(iteration,"/",it_total)
            norm_diff = np.linalg.norm(expm(A) - B)
            tqdm.write(f"Norm diff: {norm_diff}")
            if norm_diff<1e-6:
              break
    norm_diff = np.linalg.norm(expm(A) - B)
    tqdm.write(f"Final time: {t}, Norm diff: {norm_diff}")
    
    return A

## Exemples

In [27]:
#mapping B
B=np.diag([-1.,-1.])

#Données
x=np.array([[1,0.2],[0.5,2]],dtype=float)
y=np.array([B@x_i for x_i in x])

#calcul de X et S
X=calcul_X(x)
S=calcul_Y(x,y)


#initialisation
A_init=np.random.randn(2,2)

In [32]:
#calcul de A(s)
A_final=compute_A(A_init,X,S,x,y,0.1,100,B)

50 / 1000
Norm diff: 0.5210810212092581
100 / 1000
Norm diff: 9.472971168376393e-07
Final time: 9.99999999999998, Norm diff: 9.472971168376393e-07


In [33]:
#Affichage de A et exp(A)
printArray(A_final)
printArray(expm(A_final))

   1.565    3.498 
  -3.522   -1.565 
  -1.000    0.000 
  -0.000   -1.000 


## Si X et l'initialisation sont diagonaux, A(s) reste diagonal

In [34]:
#mapping B
B=np.diag([-1.,-1.])

#Données
x=np.array([[1,0],[0,2]],dtype=float)
y=np.array([B@x_i for x_i in x])

#calcul de X et S
X=calcul_X(x)
S=calcul_Y(x,y)


#initialisation
A_init=np.diag([1.,3.])

In [35]:
A_final=A_init.copy()

In [37]:
#check qu'on reste diagonal
A_final-= 0.05 * grad_matrix(A_final.copy(),X,S)
print(A_final)
print(np.linalg.norm(expm(A_final)-B))

[[-1.09138033e-01  0.00000000e+00]
 [ 0.00000000e+00 -1.66405732e+02]]
2.1440887982498373
